In [5]:
# obtain average statistical values
import pandas as pd
import numpy as np
import os.path as pathlib
from glob import glob
from scipy import stats as st
import scipy
from copy import deepcopy

cal_option = 'before_imputation'
cal_option = 'after_imputation'

def get_imputated_dataset(all_csv_file_list):
    numerical_param_set = ['param2', 'param3', 'param4', 'param5', 'param6', 'param7']
    categorial_param_set = ['param1', 'param8', 'param9', 'param10', 'param11', 'param12']
    all_csv_data = [pd.read_csv(_) for _ in all_csv_file_list]
    numerical_param_values_all = list()
    for curr_param in  numerical_param_set:
        numerical_param_values_curr = list()
        for curr_csv_data in all_csv_data:
            numerical_param_values_curr.append(curr_csv_data[curr_param])
        numerical_param_values_all.append(np.array(numerical_param_values_curr))
    numerical_param_values_all = np.mean(numerical_param_values_all, axis=1)

    categorial_param_values_all = list()
    for curr_param in categorial_param_set:
        categorial_param_values_curr = list()
        for curr_csv_data in all_csv_data:
            categorial_param_values_curr.append(curr_csv_data[curr_param])
        categorial_param_values_all.append(np.array(categorial_param_values_curr))
    categorial_param_values_all = np.array(categorial_param_values_all)
    categorial_param_values_all = categorial_param_values_all.astype(np.int64)
    categorial_param_values_all = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=categorial_param_values_all)

    new_csv_data = deepcopy(all_csv_data[0])
    for idx, name in enumerate(numerical_param_set):
        new_csv_data.drop(name, axis = 1, inplace = True)
        new_csv_data[name] = numerical_param_values_all[idx]
    for idx, name in enumerate(categorial_param_set):
        new_csv_data.drop(name, axis = 1, inplace = True)
        new_csv_data[name] = categorial_param_values_all[idx]
    return new_csv_data
    

def get_internal_csv_list(cal_option):
    root_dir = '/ssd/kangdang/mmc_project_2023_submission/main_exp_code/step1_preprocessing_redo'
    if cal_option == 'before_imputation':
        random_csv_list = [pathlib.join(root_dir, 'preprocessed_data', 'random_processed_final_csv', 'train_table_orig_random_12_vars_with_nan_intermediate_table_patient_level.csv'), 
                           pathlib.join(root_dir, 'preprocessed_data', 'random_processed_final_csv', 'val_table_orig_random_12_vars_with_nan_intermediate_table_patient_level.csv'),
                           pathlib.join(root_dir, 'preprocessed_data', 'random_processed_final_csv', 'test_table_orig_random_12_vars_with_nan_intermediate_table_patient_level.csv')]
        random_csv_content_list = [pd.read_csv(_) for _ in random_csv_list]
    if cal_option == 'after_imputation':
        cls_type_list = ['train', 'val', 'test']
        numerical_param_set = ['param2', 'param3', 'param4', 'param5', 'param6', 'param7']
        categorial_param_set = ['param1', 'param8', 'param9', 'param10', 'param11', 'param12']
        random_csv_content_list = list()
        for cls_type in cls_type_list:
            print(cls_type)
            all_csv_file_list = glob(pathlib.join(root_dir, 'preprocessed_data/random_processed_final_csv/fusion_mi', cls_type + '*' + '_mi_' + '*.csv'))
            new_csv_data = get_imputated_dataset(all_csv_file_list)
            random_csv_content_list.append(new_csv_data)
    return random_csv_content_list


def get_external_csv_list(csv_option):
    if csv_option == 'before_imputation':
        external_test = '/ssd/kangdang/mmc_project_2023_submission/external_test_code/fusion/external_test_fusion_data_after_iq_processed_fixed.csv'
        external_test = pd.read_csv(external_test)
        return [external_test] #external_test
    if csv_option == 'after_imputation':
        all_csv_file_list = glob('/ssd/kangdang/mmc_project_2023_submission/external_test_code/fusion/external_test_mice_imputation_*.csv')
        new_csv_data = get_imputated_dataset(all_csv_file_list)
        return [new_csv_data]

def get_internal_csv_list_main(csv_file_list, param_idx, ignore_nan=True):
    param_transfer_dict = dict()
    param_transfer_dict['param2'] = [54.29, 11.60]
    param_transfer_dict['param3'] = [77.34, 11.34]
    param_transfer_dict['param4'] = [131.40, 19.06]
    param_transfer_dict['param5'] = [80.826, 11.848]
    param_transfer_dict['param6'] = [25.62, 3.77]
    param_transfer_dict['param7'] = [6.80, 6.71]
    param_transfer_dict['param35'] = [1634.14, 349.12]
    
    csv_list = list()
    for csv_file in csv_file_list:
        csv_list.append(csv_file)
    curr_csv = pd.concat(csv_list)
    
    param_str = 'param35'
    curr_param_values = np.array(list(curr_csv[param_str]))
    curr_mean, curr_std = param_transfer_dict[param_str]
    curr_param_values = curr_param_values * curr_std + curr_mean
    param_str = 'param99'
    curr_csv[param_str] = curr_param_values>=1400
    
    param_str = 'param' + str(param_idx + 1)
    print(param_str)
    if param_str not in param_transfer_dict.keys():
        print(curr_csv[param_str].value_counts())
        print(curr_csv[param_str].value_counts(normalize=True))
        missing_value_no = int(np.sum(curr_csv[param_str].isnull()))
        print('missing: ' + str(missing_value_no) + ', perc=' + str(round(float(missing_value_no/len(curr_csv)),3)))
        if ignore_nan:
            isnull = curr_csv[param_str].isnull()
            result_data = curr_csv[param_str][~isnull]
        return ["categorical", list(result_data)]
    else:
        curr_param_values = np.array(list(curr_csv[param_str]))
        curr_mean, curr_std = param_transfer_dict[param_str]
        curr_param_values = curr_param_values * curr_std + curr_mean
        print('mean is : ' + str(round(np.nanmean(curr_param_values), 2)) + ', std is : ' + str(round(np.nanstd(curr_param_values), 2)))
        missing_value_no = int(np.sum(curr_csv[param_str].isnull()))
        print('missing: ' + str(missing_value_no) + ', perc=' + str(round(float(missing_value_no/len(curr_csv)),3)))
        if ignore_nan:
            isnull = curr_csv[param_str].isnull()
            result_data = curr_csv[param_str][~isnull]
        curr_param_values = np.array(list(result_data))
        curr_mean, curr_std = param_transfer_dict[param_str]
        curr_param_values = curr_param_values * curr_std + curr_mean
        return ["numerical", list(curr_param_values)]
    
def get_external_csv_list_main(csv_file_list, param_idx, ignore_nan=True):
    param_transfer_dict = dict()
    param_transfer_dict['param2'] = [54.29, 11.60]
    param_transfer_dict['param3'] = [77.34, 11.34]
    param_transfer_dict['param4'] = [131.40, 19.06]
    param_transfer_dict['param5'] = [80.826, 11.848]
    param_transfer_dict['param6'] = [25.62, 3.77]
    param_transfer_dict['param7'] = [6.80, 6.71]
    param_transfer_dict['param35'] = [0.0, 1.0]
    
    csv_list = list()
    for csv_file in csv_file_list:
        csv_list.append(csv_file)
    curr_csv = pd.concat(csv_list)
    
    param_str = 'param35'
    curr_param_values = np.array(list(curr_csv[param_str]))
    curr_mean, curr_std = param_transfer_dict[param_str]
    curr_param_values = curr_param_values * curr_std + curr_mean
    param_str = 'param99'
    curr_csv[param_str] = curr_param_values>=1400
    
    param_str = 'param' + str(param_idx + 1)
    print(param_str)
    if param_str not in param_transfer_dict.keys():
        print(curr_csv[param_str].value_counts())
        print(curr_csv[param_str].value_counts(normalize=True))
        missing_value_no = int(np.sum(curr_csv[param_str].isnull()))
        print('missing: ' + str(missing_value_no) + ', perc=' + str(round(float(missing_value_no/len(curr_csv)),3)))
        if ignore_nan:
            isnull = curr_csv[param_str].isnull()
            result_data = curr_csv[param_str][~isnull]
        return ["categorical", list(result_data)]
    else:
        curr_param_values = np.array(list(curr_csv[param_str]))
        curr_mean, curr_std = param_transfer_dict[param_str]
        curr_param_values = curr_param_values * curr_std + curr_mean
        print('mean is : ' + str(round(np.nanmean(curr_param_values), 2)) + ', std is : ' + str(round(np.nanstd(curr_param_values), 2)))
        missing_value_no = int(np.sum(curr_csv[param_str].isnull()))
        print('missing: ' + str(missing_value_no) + ', perc=' + str(round(float(missing_value_no/len(curr_csv)),3)))
        if ignore_nan:
            isnull = curr_csv[param_str].isnull()
            result_data = curr_csv[param_str][~isnull]
        curr_param_values = np.array(list(result_data))
        curr_mean, curr_std = param_transfer_dict[param_str]
        curr_param_values = curr_param_values * curr_std + curr_mean
        return ["numerical", list(curr_param_values)]



In [8]:
from collections import Counter

def get_contingency_table(tr_val_value, te_value):
    tr_val_dict = Counter(tr_val_value)
    te_dict = Counter(te_value)
    contingency_table = list()
    tr_val_dict_keys_sorted = list(np.sort(list(tr_val_dict.keys())))
    for tr_key_idx in tr_val_dict_keys_sorted:
        tr_value = tr_val_dict[tr_key_idx]
        te_value = te_dict[tr_key_idx]
        contingency_table.append((tr_value, te_value))
    return contingency_table

def get_results_stat_test(interal_csv_data, external_csv_data):
    for i in list(range(12)) + [34] + [98]:
        print('\n')
        print('param' + str(i + 1))
        print('tr_val')
        tr_val = get_internal_csv_list_main([interal_csv_data[0], interal_csv_data[1]], i)
        print('test')
        te =  get_external_csv_list_main(external_csv_data, i)
        print('stat_kstest')
        print(tr_val[0])
        if tr_val[0] == "numerical":
            norm_test_result = scipy.stats.kstest(tr_val[1], cdf="norm")
            if norm_test_result.pvalue > 0.05:
                print('normal distribution')
                result = scipy.stats.ttest_ind(tr_val[1], te[1])
            else:
                print('not normal distribution')
                result = scipy.stats.ranksums(tr_val[1], te[1])
            print(result)
        if tr_val[0] == "categorical":
            if i == 0: 
                # we convert gender variable to a binary variable
                tr_val[1] = np.array(tr_val[1]) - 1
                te[1] = np.array(te[1]) - 1
            contingency_table = get_contingency_table(tr_val[1], te[1])
            print(contingency_table)
            result = scipy.stats.chi2_contingency(contingency_table, correction=False)
            print(result)

In [9]:
cal_option = 'before_imputation'
interal_csv_data =  get_internal_csv_list(cal_option)
external_csv_data = get_external_csv_list(cal_option)
get_results_stat_test(interal_csv_data, external_csv_data)



param1
tr_val
param1
param1
1    17075
2    12006
Name: count, dtype: int64
param1
1    0.587153
2    0.412847
Name: proportion, dtype: float64
missing: 0, perc=0.0
test
param1
param1
1.0    2658
2.0    1241
Name: count, dtype: int64
param1
1.0    0.681713
2.0    0.318287
Name: proportion, dtype: float64
missing: 0, perc=0.0
stat_kstest
categorical
[(17075, 2658), (12006, 1241)]
Chi2ContingencyResult(statistic=127.91419586257467, pvalue=1.1720191524502398e-29, dof=1, expected_freq=array([[17400.10227411,  2332.89772589],
       [11680.89772589,  1566.10227411]]))


param2
tr_val
param2
mean is : 53.27, std is : 11.22
missing: 0, perc=0.0
test
param2
mean is : 52.52, std is : 11.74
missing: 0, perc=0.0
stat_kstest
numerical
not normal distribution
RanksumsResult(statistic=2.5384802461322926, pvalue=0.011133508796416737)


param3
tr_val
param3
mean is : 77.53, std is : 11.26
missing: 739, perc=0.025
test
param3
mean is : 78.6, std is : 10.09
missing: 805, perc=0.206
stat_kstest
numeric

In [10]:
cal_option = 'after_imputation'
interal_csv_data =  get_internal_csv_list(cal_option)
external_csv_data = get_external_csv_list(cal_option)
get_results_stat_test(interal_csv_data, external_csv_data)

train
val
test


param1
tr_val
param1
param1
0    17075
1    12006
Name: count, dtype: int64
param1
0    0.587153
1    0.412847
Name: proportion, dtype: float64
missing: 0, perc=0.0
test
param1
param1
1    2658
2    1241
Name: count, dtype: int64
param1
1    0.681713
2    0.318287
Name: proportion, dtype: float64
missing: 0, perc=0.0
stat_kstest
categorical
[(17075, 0), (12006, 2658)]
Chi2ContingencyResult(statistic=3377.902733481982, pvalue=0.0, dof=1, expected_freq=array([[15645.04473991,  1429.95526009],
       [13435.95526009,  1228.04473991]]))


param2
tr_val
param2
mean is : 53.27, std is : 11.22
missing: 0, perc=0.0
test
param2
mean is : 52.52, std is : 11.74
missing: 0, perc=0.0
stat_kstest
numerical
not normal distribution
RanksumsResult(statistic=2.5328876903296473, pvalue=0.01131271924422569)


param3
tr_val
param3
mean is : 77.51, std is : 11.13
missing: 0, perc=0.0
test
param3
mean is : 78.24, std is : 9.18
missing: 0, perc=0.0
stat_kstest
numerical
not normal distributio